# TASK 4
Raggruppamento di ristoranti a partire da un nuovo locale sulla base delle sue caratteristiche oppure del voto degli utenti

In [196]:

from sklearn.neighbors import NearestNeighbors
from sklearn.model_selection import train_test_split
from sklearn.decomposition import TruncatedSVD

from libraries.dataset import Dataset
import libraries.data_handler as data_handler
import pandas as pd
import numpy as np
import ast

## Data retrieving
Ottenimento dei dati relativi alle recensioni, bilanciati sulla base delle stelle, al fine di ottenere lo stesso numero di review per ogni possibile valutazione (da 1 a 5).
In questo specifico caso, sono richiesti 20'000 campioni per ogni tipo di classe (per un totale di 100'000 campioni).

L'oggetto `review_data` contiene tre field relativi ai subdataset da utilizzare nel progetto:
- `train_data` = tupla contentente i dati ed i target per il training
- `val_data` = tupla contentente i dati ed i target per la validazione
- `test_data` = tupla contentente i dati ed i target per il testing

Alla prima esecuzione, i tre diversi subset sono memorizzati sottoforma di file csv, in modo da evitare la riesecuzione del codice di splitting dei dataset durante le successive esecuzioni.  

In [ ]:
review_data = Dataset('review', 'stars')

# 20_000 elements for each class
review_data.split(['user_id', 'business_id', 'text'], 'stars', n_samples=20_000)

L'oggetto `business_data` contiene tutti le informazioni relative alle attività all'interno di tre stati degli Stati Uniti.

In particolare, le feature che compongono il dataset sono il business id, il nome, l'indirizzo, le categorie a cui appartiene un'attività, gli attributi (parcheggio, fascia di prezzo, ecc.) e il voto medio dell'attività in stelle.

In [ ]:
business_data = pd.DataFrame(data_handler.load_dataset('business'))
business_data = business_data[(business_data['state']=='GA') | (business_data['state'] == 'FL') | (business_data['state'] == 'TX')].reset_index()
business_data = business_data[['business_id','name','address', 'categories', 'attributes','stars']]

Selezione dei soli ristoranti e salvataggio nell'oggetto `restaurants_data`.

In [5]:
restaurants_data = business_data[business_data['categories'].str.contains('Restaurant.*')==True].reset_index()

Definizione delle funzioni per l'estrazione delle chiavi contenute all'interno di dizionari annidati nel dizionario della feature `attributes` e per la conversione di una stringa in dizionario.

In [6]:
# Function that extract keys from the nested dictionary
def extract_keys(attr, key):
    if attr == None:
        return "{}"
    if key in attr:
        return attr.pop(key)

# convert string to dictionary
def str_to_dict(attr):
    if attr != None:
        return ast.literal_eval(attr)
    else:
        return ast.literal_eval("{}")   

In [7]:
def to_Dict(attribute):
    restaurants_data[attribute] = restaurants_data.apply(lambda x: str_to_dict(extract_keys(x['attributes'], attribute)), axis=1)

In [8]:
to_Dict('BusinessParking')
to_Dict('Ambience')
to_Dict('GoodForMeal')
to_Dict('Dietary')
to_Dict('Music')

Concatenazione delle serie relative alle feature che formeranno il nuovo dataset e one-hot encoding di tali serie.

In [ ]:
df_attr = pd.concat([ restaurants_data['attributes'].apply(pd.Series), restaurants_data['BusinessParking'].apply(pd.Series),
                    restaurants_data['Ambience'].apply(pd.Series), restaurants_data['GoodForMeal'].apply(pd.Series), 
                    restaurants_data['Dietary'].apply(pd.Series)], axis=1)
df_attr_dummies = pd.get_dummies(df_attr)

In [10]:
df_categories_dummies = pd.Series(restaurants_data['categories']).str.get_dummies(',')
df_categories_dummies

,Acai Bowls,Accessories,Active Life,Adult Education,Adult Entertainment,Afghan,African,Airport Lounges,Airport Shuttles,Airports,...,Wedding Planning,Whiskey Bars,Wholesale Stores,Wholesalers,Wine Bars,Wine Tasting Room,Wineries,Women's Clothing,Wraps,Yelp Events
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19300,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
19301,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
19302,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
19303,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
result = restaurants_data[['name','stars']]
result

,name,stars
0,Sister Honey's,4.5
1,Everything POP Shopping & Dining,3.0
2,RaceTrac,3.5
3,Cascade Restaurant,3.5
4,El Pollo Rey,5.0
...,...,...
19300,LongHorn Steakhouse,4.0
19301,Saigon Noodle & Grill,4.5
19302,Maudie’s Hill Country,3.0
19303,Mama's Cocina Latina,3.0


Dataset finale ottenuto dalla concatenazione di attributi, categorie e ristoranti.

Il dataset finale contiene attributi e categorie come feature trasformate con il one-hot encoding del passo precedente. Inoltre, le stelle venogno convertite in interi per agevolarne l'utilizzo nell'addestramento dei modelli.

In [12]:
df_final = pd.concat([df_attr_dummies, df_categories_dummies, result], axis=1)
df_final.drop('Restaurants',inplace=True,axis=1)
df_final['stars'] = df_final['stars'].astype(int)

## Content Based Filtering

### KNN
Addestramento del classificatore di tipo KNN sul dataset finale.

E' stato scelto di addestrare il modello impostando il valore dei k vicini a 5 ed è stato effettuato il fitting sull'80% dei campioni del dataset originale.

In [188]:
X_train_knn, X_test_knn, y_train_knn, y_test_knn = train_test_split(df_final.iloc[:,:-2], df_final['stars'], test_size=0.2, random_state=1)

In [190]:
knn = NearestNeighbors(n_neighbors=5)
knn.fit(X_train_knn, y_train_knn)

NearestNeighbors()

Visualizzazione del nome del ristorante scelto per la fase di testing.

In [197]:
# look at the restaurant name from the last row.
print("Test set (Restaurant name): ", df_final['name'].values[-1])

Test set (Restaurant name):  Zora Grille


Definizione del test e del validation set rispettivamente come l'ultimo sample del dataset originale e il dateset originale esclusa l'ultima riga.

In [192]:
# test set from the df_final table (only last row)
test_set = df_final.iloc[-1:,:-2]

# validation set from the df_final table (exclude the last row)
X_val =  df_final.iloc[:-1,:-2]
y_val = df_final['stars'].iloc[:-1]

Fitting del k-nn sul validation set.

In [193]:
# fit model with validation set
n_knn = knn.fit(X_val, y_val)

Calcolo dei vicini e dunque dei ristoranti da suggerire rispetto al test set, ovvero l'ultimo sample del dataset originale.

In [194]:
distances, indeces =  n_knn.kneighbors(test_set)

final_table = pd.DataFrame(n_knn.kneighbors(test_set)[0][0], columns = ['distance'])
final_table['index'] = n_knn.kneighbors(test_set)[1][0]
final_table.set_index('index')

,distance
index,
2390,3.316625
5135,3.464102
9057,3.464102
2605,3.605551
18103,3.605551


Visualizzazione dei ristoranti suggeriti sulla base delle caratteristiche del ristorante contenuto nel test set.

In [195]:

result = final_table.join(df_final,on='index')
result[['distance','index','name','stars']]

,distance,index,name,stars
0,3.316625,2390,Mezza,4
1,3.464102,5135,Royal Indian Cuisine,4
2,3.464102,9057,Ray's Rio Bravo,2
3,3.605551,2605,Nicola's Restaurant,4
4,3.605551,18103,Punjab Indian Restaurant,2


## Collaborative Based Filtering

## SVD
Utilizzo dell'SVD, una particolare fattorizzazione di matrice basata sull'utilizzo di autovalori e autovettori.

Viene utilizzata una combinazione del dataset delle review e dei ristoranti al fine di ottenere una matrice. A partire da tale matrice si otterranno poi i ristoranti che risultano avere una correlazione particolarmente forte con il ristorante in input.

In [32]:
restaurants = restaurants_data[['business_id', 'name', 'address']]
reviews = review_data.train_data[0]
reviews['stars'] = review_data.train_data[1]
combined_business_data = pd.merge(reviews, restaurants, on='business_id')
combined_business_data

,user_id,business_id,text,stars,name,address
0,I0V_eIXYmT-8t24oqSH6Ag,A2z__2uOtiVcAXBXDOAvwQ,I wouldn't come here at all anymore aside from...,2,Wing Shack,4650 E Michigan St
1,w5V4GY5Qn8WcmgZG6jEXUQ,A2z__2uOtiVcAXBXDOAvwQ,Not bad at all. This is a neighborhood bar wit...,3,Wing Shack,4650 E Michigan St
2,zyUZ2DW39g8gNkbyVRYmow,A2z__2uOtiVcAXBXDOAvwQ,Stopped by here because I was craving for some...,3,Wing Shack,4650 E Michigan St
3,3RZ16e8z4gplxUPoQs0STg,A2z__2uOtiVcAXBXDOAvwQ,biggest piece of shit ever...\nthey used uber ...,1,Wing Shack,4650 E Michigan St
4,mprt6meg7XghdUwSZCu21Q,zFaHweOJ40jjtvpGTjlspw,I've been a regular at Uchi Houston so I was e...,2,Uchi,801 South Lamar Blvd
...,...,...,...,...,...,...
27923,FBsFuczGvG4nkrZpXs47hg,5a_jB7LruuScswVL5kgMuA,"I have gone twice, so far, with friends to the...",5,Nancy's Sky Garden,"6448 East Highway 290, Ste A-100"
27924,Zq_i9_--tP3tlYUXh3aj7g,d-nWgOx9rJco8dtlA1mMvA,"Meh, for all the amazing things I have heard a...",2,Highland Bakery,224 Uncle Heinie Way
27925,DPZeq2mZj9oFgf2TuMC5_w,mPEBLunXpp-M8wXAsbIXrA,Really good pizza! We ordered through Uber Ea...,4,Flippers Pizzeria,3216 Rolling Oaks Blvd
27926,QqxR7bNrfckOFeULFhDBtA,y_gg6SWvqRUChXaYFsip6Q,I went to lunch with a co-worker as we knew th...,1,BurgerIM Midtown,"933 Peachtree St NE, Ste 951"


Si visualizzano i primi cinque ristoranti che hanno avuto il numero più alto di stelle da parte dell'utente.

In [33]:
combined_business_data.groupby('business_id')['stars'].count().sort_values(ascending=False).head()

business_id
bZiIIUcpgxh8mpKMDhdqbA    45
sPhPI3B6tvcJIULhICr-Pg    44
CxQ1m2iY4wQpXC64tSfWgQ    43
WkN8Z2Q8gbhjjkCt8cDVxg    43
MGzro82Fi4LYvc86acoONQ    42
Name: stars, dtype: int64

Si seleziona il ristorante che verrà poi passato in input al recommendation system.

In [34]:
Filter = combined_business_data['business_id'] == 'bZiIIUcpgxh8mpKMDhdqbA'
print("Name: ", combined_business_data[Filter]['name'].unique())
print("Address:", combined_business_data[Filter]['address'].unique())

Name:  ['Hopdoddy Burger Bar']
Address: ['1400 S Congress Ave, Ste A190']


Calcolo della matrice le cui righe corrispondono agli utenti, le colonne ai ristorante ed ogni cella rappresenta il numero di stelle che l'utente ha assegnato ad ogni ristorante.

In [35]:
rating_crosstab = combined_business_data.pivot_table(values='stars', index='user_id', columns='name', fill_value=0)
rating_crosstab.head()

name,# 19,'Ohana,081 Wood Fired Pizza,10 Degrees South,100 Montaditos,100 Pizzitas,1000 Degrees Neapolitan Pizzeria,1000 Degrees Pizza Salad Wings,101 By Teahaus,101 Steak,...,la Barbecue,la Madeleine French Bakery & Cafe,la vita pizza & pasta,laV,mmmpanadas,nati's southern seafood boil,planet bombay Indian cuisine,sandoitchi Pop Up,zpizza,ñoños tacos
user_id,,,,,,,,,,,,,,,,,,,,,
--JgAxm4jQ3GPgOp6BjGsw,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
--K6YIslIaE0O7TDcSraSg,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
--KTS4VPl9G-_FDWst1SQg,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
--Lm6zODOhH_9bLtRQfz4w,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
--d97ty7k0X_wt4uVCS45Q,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Fitting dell'SVD sulla matrice ottenuta.

In [66]:
SVD = TruncatedSVD(n_components=12, random_state=17)
result_matrix = SVD.fit_transform(rating_crosstab.values.T)

Calcolo matrice dei coefficienti di correlazione a partire dalla matrice ottenuta dall'SVD.

In [69]:
corr_matrix = np.corrcoef(result_matrix)

Ricerca dell'indice relativo al ristorante con il maggior numero di stelle.

In [70]:
restaurant_names = rating_crosstab.columns
restaurants_list = list(restaurant_names)

popular_rest = restaurants_list.index('Hopdoddy Burger Bar')
print("index of the popular restaurant: ", popular_rest) 

index of the popular restaurant:  2852


Selezione degli indici dei ristoranti che più si avvicinano al ristorante in input al recommendation system sulla base dei voti ottenuti dall'utente.

In [118]:
corr_popular_rest = corr_matrix[popular_rest]
corrIndices = [i for i, x in enumerate((corr_popular_rest <= 1.0) & (corr_popular_rest > 0.9)) if x]
corrIndices

[948, 1809, 2257, 2397, 2852, 2973, 5114, 5563, 5811, 6230, 6653, 7112]

Visualizzazione dei 10 ristoranti suggeriti dall'SVD con la relativa correlazione.

In [136]:
recommendedRest = [(restaurant_names[i], corr_matrix[popular_rest, i]) for i in corrIndices]
recommendedRest.sort(key=lambda x: x[1], reverse=True)
recommendedRest.pop(0)
recommendedRest[:10]

[('Doghouse', 0.9999955245780303),
 ('Fogo De Chão Brazilian Steakhouse', 0.9999942452978231),
 ('Zaviya Grill', 0.9999122444240708),
 ('Tumble22', 0.9998342999940556),
 ('Independence Fine Foods', 0.9997540124544847),
 ('Social House Orlando', 0.9995505478815057),
 ("Rio's Brazilian Cafe", 0.9994173499828607),
 ('Sway', 0.9928870187377924),
 ('Bun Belly', 0.9915646397639292),
 ('The Funkadelic', 0.9519420018894759)]